In [1]:
import gc
gc.collect()

import sys
sys.path.insert(0, '../')
import logging
logging.basicConfig(level=logging.ERROR)

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from cryptotrader.exchange_api.poloniex import Poloniex
from cryptotrader.envs.trading import BacktestDataFeed, BacktestEnvironment
from cryptotrader.envs.utils import convert_and_clean, convert_to
from cryptotrader.agents import apriori
from cryptotrader.utils import array_normalize

from time import time, sleep

from bokeh.io import output_notebook
from jupyterthemes import jtplot
output_notebook()
jtplot.style()
%matplotlib inline
%load_ext line_profiler

import matplotlib.pyplot as plt

Loading BokehJS ...

In [ ]:
obs_steps = 300
period = 30
pairs = ["USDT_BTC", "USDT_ETH", "USDT_LTC", "USDT_XRP"]
init_funds = {"BTC":'0.00000000', "ETH":'0.00000000', "LTC":'0.00000000', "XRP":'0.00000000', "USDT":'100.00000000'}

tapi = Poloniex()
tapi = BacktestDataFeed(tapi, period, pairs, init_funds)
tapi.download_data(end=datetime.timestamp(datetime.utcnow() - timedelta(days=100)),
                       start=datetime.timestamp(datetime.utcnow() - timedelta(days=200)))

env = BacktestEnvironment(period, obs_steps, tapi, 'backtest_env')
env.add_pairs(pairs)
env.fiat = 'USDT'
obs = env.reset()

INFO:backtest_env:[Trading Environment initialization]
Trading Environment Initialized!



In [ ]:
nb_steps = 100
batch_size = 32
nb_max_episode_steps = 7

mom = apriori.MomentumTrader()
fib = apriori.HarmonicTrader()
pamr = apriori.PAMRTrader()

mom.set_params(**{'mean_type': 'kama', 'ma1': 7, 'ma2': 177, 'std_span': 100, 'std_weight_down': 1.5866992187499998, 'std_weight_up': 0.67654296875} )
fib.set_params(**{'err_allowed': .0563, 'peak_order': 12})
pamr.set_params(**{'variant': 'PAMR1', 'C': 1685, 'sensitivity': 0.031357896566833805})

agent = apriori.FactorTrader([mom, fib, pamr])
opt_params, info = agent.fit(env, nb_steps, batch_size=batch_size, nb_max_episode_steps=nb_max_episode_steps)
print('\n', opt_params, '\n', env.status)
agent.test(env, verbose=True)
env.plot_results();

Optimizing model for 100 steps with batch size 32...


In [ ]:
tapi.download_data(end=datetime.timestamp(datetime.now()),
                       start=datetime.timestamp(datetime.now() - timedelta(days=100)))
agent.test(env, verbose=True)
env.plot_results();